<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [196]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import get_custom_objects
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [197]:
df=pd.read_csv("/content/updated_dataset.csv")

In [198]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])




In [199]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [200]:

X = df[features].values
y = df[targets].values

In [201]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [202]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)

In [203]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)

In [204]:

def custom_loss(y_true, y_pred):
    batch_size = tf.shape(y_true)[0]
    batch_hours = tf.gather(hours_tensor, tf.range(batch_size))

    nighttime_mask = tf.logical_or(tf.less_equal(batch_hours, 8), tf.greater_equal(batch_hours, 18))
    nighttime_mask = tf.cast(nighttime_mask, tf.float32)

    mse_loss = tf.keras.losses.MSE(y_true, y_pred)
    penalty_factor = 50.0
    weighted_loss = mse_loss * (1 + (penalty_factor - 1) * nighttime_mask)

    return tf.reduce_mean(weighted_loss)



In [205]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(targets), activation='linear')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [206]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss, metrics=['mae'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [207]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 3344716.7500 - mae: 195.9732 - val_loss: 1638979.6250 - val_mae: 119.8947
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1596641.7500 - mae: 122.4412 - val_loss: 1335420.0000 - val_mae: 105.9947
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 1367395.6250 - mae: 103.2745 - val_loss: 1261145.1250 - val_mae: 95.3853
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1344152.1250 - mae: 100.4029 - val_loss: 1270735.5000 - val_mae: 91.3668
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1330391.3750 - mae: 98.3579 - val_loss: 1176068.0000 - val_mae: 91.5103
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1241067.0000 - mae: 93.4106 - val_loss: 1196822.0000 - val_mae: 93.4830
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1221959.1250 - mae: 94.1767 - val_loss: 1166642.7500 - val_mae: 87.7225
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1182804.00

In [208]:
# Predict future solar irradiance for all three locations
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [209]:
# Convert predictions into a DataFrame
predictions_df = timestamps_test.copy()

# Assign each column separately
predictions_df[['DNI_A']] = y_pred

In [210]:
# Print first 100 predictions with date and hour
print(predictions_df.head(100))
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)  # Saves to a CSV]

      Month  Day  Hour       DNI_A
6056      9   10     8  177.115738
5556      8   20    12  356.378906
5990      9    7    14  339.641937
7674     11   16    18  135.529770
3319      5   19     7  101.279129
...     ...  ...   ...         ...
6006      9    8     6   10.508432
8063     12    2    23   -0.647378
14        1    1    14  449.262390
5306      8   10     2    0.274714
8524     12   22     4    0.587974

[100 rows x 4 columns]


In [211]:
# Save the model
model.save("solar_irradiance_predictions.keras")

# Load the model (now it recognizes custom_loss)
model = keras.models.load_model("solar_irradiance_predictions.keras", custom_objects={"custom_loss": custom_loss})


In [212]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [213]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-213-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [214]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [215]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [216]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[ 0.          0.60099836  1.91278729 ...  0.18181818  4.70457419
   0.33333333]
 [ 0.04347826  0.66020201  1.73382569 ...  0.18181818 -0.78169312
   0.11111111]
 [ 0.08695652  0.6400745   0.57891126 ...  0.18181818  6.53990555
   0.        ]
 ...
 [ 0.91304348  0.6458473   0.81572072 ...  0.18181818  5.32733798
   0.11111111]
 [ 0.95652174  0.65107552  0.43599095 ...  0.18181818  5.10722776
   0.33333333]
 [ 1.          0.66136903  1.85614991 ...  0.18181818  3.24341612
   0.        ]]


In [217]:
#  Make predictions
y_future_scaled = model.predict(X_future_scaled)

#  Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled)
#  Set negative irradiance values to 0
y_future[y_future < 15] = 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [218]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      27.792546  1048.383619          0.375229   
1    1      3     1      29.722586  1043.014771          0.100915   
2    1      3     2      29.066429  1008.367338          0.466995   
3    1      3     3      27.423631  1012.450179          0.197352   
4    1      3     4      29.358945  1040.076105          0.121038   
5    1      3     5      28.306957  1035.630437          0.196774   
6    1      3     6      27.025529  1046.318104          0.265925   
7    1      3     7      27.614039  1036.933264          0.308844   
8    1      3     8      29.992946  1024.214056          0.206116   
9    1      3     9      28.137253  1034.458245          0.122183   

   Cloud Type_A       DNI_A  
0             3    0.000000  
1             1    0.000000  
2             0   27.338976  
3             0    0.000000  
4             2   15.053878  
5             0   55.526337  
6             1  135.328049  
